In [ ]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import Estimate
import figure 
from EconModel import cpptools
import pickle

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)


plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings

do_compile = True
threads = 20
T = 20
#cpptools.setup_nlopt(folder='cppfuncs/', do_print = True) #install nlopt

import warnings
warnings.filterwarnings("ignore")



#Normal 9000
#model_FC 14,000
#model_LC 3500
#model_NC 3500 

#REST FEAS 9000
#model_FC 20,000  
#model_LC 6000 
#model_NC 6000 





# Load the model

In [ ]:
name = 'model_FC'
model_name = "model_FC"
do_distr_yerror = "none"
do_yerror = 0
#do_BMI_dummy = False 
do_sep_shock = False 
#name_error = f'{do_distr_yerror}{do_yerror}std_Yerror_rest0_restOutl'
name_error = f'{do_distr_yerror}{do_yerror}std_Yerror_restr_feas'
N = 20000
NN = "NN" #NN number of observations in estimation, N er samme antal simuleringer
NNO = 9000
rest_sample_feas = True
rest_sample_nonfeas    = False
#name_error = "uni2std_Yerror"

specs = {}


#specs.update({f'model_LC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':1}}})
#specs.update({f'model_NC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':2}}})
specs.update({f'model_FC':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': False, 'simN': N,  'threads':threads,'bargaining':0}}})

# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    
    # setup model
    model = LimitedCommitmentModelClass(name=name,par=spec['par'])
    model.spec = spec

    compile_now = True if do_compile & (m==0) else False
    model.link_to_cpp(force_compile=compile_now)
    

# setup model
#model = LimitedCommitmentModelClass(name=name)    
#model.link_to_cpp(force_compile=True)


#load par and sol
pickle_file = f'output/{name}_sol.pickle'

with open(pickle_file, 'rb') as f:
    model.sol = pickle.load(f)

pickle_file = f'output/{name}_par.pickle'

with open(pickle_file, 'rb') as f:
    model.par = pickle.load(f)
    model.par.simN = N

In [ ]:
#Do 1 time estimate
data = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2, yerror = do_distr_yerror, scale_st = do_yerror )

data = Estimate.create_variable(data, model.par,gender= "w", do_estimate_wage = "est_omega", print_reg = True, shadow_value_simple=2, do_control_love = False, part_earning_simple =1, control_cons = 1, wealth_love = 1)

if rest_sample_nonfeas:  
    if model_name != "model_FC":
        # Remove those who do not update bargaining weight
        
        data['delta_log_barganing'] = data['delta_log_barganing'].mask(data['delta_log_barganing'].between(-0.001,0.001))
    

if rest_sample_feas:  
    #not at the limit
    data =data[data['omega_res_w'] < 0.1]
    data =data[data['omega_res_w'] > -0.1]
    data =data[data['omega_res_m'] < 0.1]
    data =data[data['omega_res_m'] > -0.1]

    

    data =data[data['omega_res_w_l'] < 0.1]
    data =data[data['omega_res_w_l'] > -0.1]
    data =data[data['omega_res_m_l'] < 0.1]
    data =data[data['omega_res_m_l'] > -0.1]

        
    data =data[data['omega_res_w_l2'] < 0.1]
    data =data[data['omega_res_w_l2'] > -0.1]
    data =data[data['omega_res_m_l2'] < 0.1]
    data =data[data['omega_res_m_l2'] > -0.1]

print(data)


_, wald_FC_w , wald_NC_w, _,_,  _ = Estimate.main_est2(data,gender= "w", do_estimate_wage = "est_omega", print_reg = True, shadow_value_simple=2, do_control_love = False, part_earning_simple =2, control_cons = 2, wealth_love = 1)


In [ ]:
model.par.seed = 2024
np.random.seed(model.par.seed)

no_sim = 100
wald_FC_w_pval = np.nan + np.ones((4,2,2,2,2,2,no_sim) )
wald_NC_w_pval = np.nan + np.ones((4,2,2,2,2,2,no_sim) )
wald_FC_m_pval = np.nan + np.ones((4,2,2,2,2,2,no_sim) )
wald_NC_m_pval = np.nan + np.ones((4,2,2,2,2,2,no_sim) )




coef_w = {}
coef_m = {}





N_w= np.nan + np.ones(no_sim) 
N_m= np.nan + np.ones(no_sim) 
#loop over shadow values
#for j in range(4):
#    j = j +1
for k in [1,2,3]:
    if k == 1:
        #BASELINE WITH DETAILED SHADOW
        j = 2 #nonpar shadow
        jj = 2 #nonpar controls
        jjj = 1 #par wage
        p = False #True bargaining
        ip = int(p)
        ik = 0 # wage
        do_wage = "est_omega"
        im = 0 #love
        do_love = False 
        z = False #do Heckman selection
        iz = int(z)
    if k == 2:
        #include HECKMANS selection 
        j = 2 #nonpar shadow
        jj = 2 #nonpar controls
        #jjj = 1 #par wage
        p = False #True bargaining
        ip = int(p)
        ik = 0 # wage
        do_wage = "est_omega"
        im = 0 #love
        do_love = False 
        z = True  #do Heckman selection
        iz = int(z)
    if k == 3:
        #include HECKMANS selection + love + true wage shock 
        j = 2 #nonpar shadow
        jj = 2 #nonpar controls
        #jjj = 1 #par wage
        p = False #True bargaining
        ip = int(p)
        ik = 1 # wage
        do_wage = "true_omega"
        im = 1 #love
        do_love = True 
        z = True  #do Heckman selection
        iz = int(z)
    if k == 4:
        #include HECKMANS selection + love + true wage shock + true bargaining
        j = 2 #nonpar shadow
        jj = 2 #nonpar controls
        #jjj = 1 #par wage
        p = True #True bargaining
        ip = int(p)
        ik = 1 # wage
        do_wage = "true_omega"
        im = 1 #love
        do_love = True 
        z = True  #do Heckman selection
        iz = int(z)

    for i in range(no_sim):
        print(f'k = {k}, i = {i}')
        data = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2, yerror = do_distr_yerror, scale_st = do_yerror)
        data = Estimate.create_variable(data, model.par,gender= "w", do_estimate_wage = do_wage, print_reg = False, shadow_value_simple=j, do_control_love = do_love, part_earning_simple =1, control_cons = 1,  do_true_barg = p, control_select = z)
        if rest_sample_nonfeas:  
            if model_name != "model_FC":
                # Remove those who do not update bargaining weight
                #data = data[data['delta_log_barganing'] != 0]
                
                data['delta_log_barganing'] = data['delta_log_barganing'].mask(data['delta_log_barganing'].between(-0.001,0.001))
                

                #data =data[data['delta_log_barganing_l'] > -0.1]
                #data =data[data['delta_log_barganing_l'] <  0.1]

                #data =data[data['delta_log_barganing_l2'] > -0.1]
                #data =data[data['delta_log_barganing_l2'] <  0.1]

        if rest_sample_feas:  
            #not at the limit
            data =data[data['omega_res_w'] < 0.1]
            data =data[data['omega_res_w'] > -0.1]
            data =data[data['omega_res_m'] < 0.1]
            data =data[data['omega_res_m'] > -0.1]

            

            data =data[data['omega_res_w_l'] < 0.1]
            data =data[data['omega_res_w_l'] > -0.1]
            data =data[data['omega_res_m_l'] < 0.1]
            data =data[data['omega_res_m_l'] > -0.1]

                
            data =data[data['omega_res_w_l2'] < 0.1]
            data =data[data['omega_res_w_l2'] > -0.1]
            data =data[data['omega_res_m_l2'] < 0.1]
            data =data[data['omega_res_m_l2'] > -0.1]

       
        data = Estimate.aux_est(data,model.par)
        _, wald_FC_w , wald_NC_w, wald_FC_noW_w , wald_NC_noW_w, c_w = Estimate.main_est2(data,gender= "w", do_estimate_wage = do_wage, print_reg = False, shadow_value_simple=j, do_control_love = do_love, part_earning_simple = jj, control_cons= jj,  do_true_barg = p, control_select = z)
        _, wald_FC_m , wald_NC_m, wald_FC_noW_m , wald_NC_noW_m, c_m = Estimate.main_est2(data,gender= "m", do_estimate_wage = do_wage, print_reg = False, shadow_value_simple=j, do_control_love = do_love, part_earning_simple = jj, control_cons= jj,  do_true_barg = p, control_select = z)
        wald_FC_w_pval[j-1, jj-1,ip,ik, im, iz, i] = wald_FC_w.pvalue
        wald_FC_m_pval[j-1, jj-1,ip,ik, im, iz, i] = wald_FC_m.pvalue
        wald_NC_w_pval[j-1, jj-1,ip,ik, im, iz, i] = wald_NC_w.pvalue
        wald_NC_m_pval[j-1, jj-1,ip,ik, im, iz, i] = wald_NC_m.pvalue

        #wald_FC_noW_w_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_FC_noW_w.pvalue
        #wald_FC_noW_m_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_FC_noW_m.pvalue
        #wald_NC_noW_w_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_NC_noW_w.pvalue
        #wald_NC_noW_m_pval[j-1, jj-1, jjj-1, ik, im, i] = wald_NC_noW_m.pvalue
        
        coef_w[f'shadow{j}_inc{jj}_cons{jj}_wage{ik}_love{im}_barg{ip}_select{iz}_sim{i}'] =  c_w
        coef_m[f'shadow{j}_inc{jj}_cons{jj}_wage{ik}_love{im}_barg{ip}_select{iz}_sim{i}'] =  c_m
        



In [ ]:
#SAVE PICKLE
pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_testFC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_FC_w_pval,f)
pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_testFC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_FC_m_pval,f)
pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_testNC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_NC_w_pval,f)
pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_testNC.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(wald_NC_m_pval,f)

pickle_file = f'output/TRUE{name}_m_{NN}{NNO}_S{no_sim}_{name_error}_coef.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(coef_m,f)

pickle_file = f'output/TRUE{name}_w_{NN}{NNO}_S{no_sim}_{name_error}_coef.pickle'
with open(pickle_file, 'wb') as f:
    pickle.dump(coef_w,f)



In [ ]:
stop